# New Section

In [1]:
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime }")
# # Load the TensorBoard notebook extension
# %load_ext tensorboard
import tensorflow as tf
import os

print(f"TensorFlow version: {tf.__version__}")

Notebook last run (end-to-end): <class 'datetime.datetime'>


TensorFlow version: 2.10.1


In [2]:
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [3]:
walk_through_dir("/content/plant_dataset.rar")

In [4]:
import zipfile

def unzip_data(filename, extract_to="."):
    """
    Unzips a file into the specified directory.

    Args:
        filename (str): The filepath of the ZIP file to be unzipped.
        extract_to (str): The directory where the contents will be extracted.
                          Defaults to the current directory.
    """
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(extract_to)



In [6]:

# # Example usage:
# zip_file = "plant_dataset.zip"
# unzip_data(zip_file, "extracted_data")  # Extracts to the "extracted_data" directory

In [7]:
#from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir
walk_through_dir("/content/extracted_data/plant_dataset")
# Create training and test directories
train_dir = "/content/extracted_data/plant_dataset/train"
test_dir = "/content/extracted_data/plant_dataset/test"

In [8]:
import tensorflow as tf
IMG_SIZE = (224,224)
train_data_leaves = tf.keras.preprocessing.image_dataset_from_directory(directory =  train_dir,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode = "categorical",
                                                                            batch_size = 32)
test_data_leaves = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical")
train_data_leaves
train_data_leaves.class_names
for images, labels in train_data_leaves.take(1):
  print(images, labels)
train_data_leaves.take(1)

NotFoundError: Could not find directory /content/extracted_data/plant_dataset/train

In [ ]:

# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)

# OLD
# base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Freeze the base model (so the pre-learned patterns remain)
base_model.trainable = False

24274472/24274472 [==============================] - 0s 0us/step


In [ ]:

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback


In [ ]:

# 3. Create inputs into the base model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. If using ResNet50V2, add this to speed up convergence, remove for EfficientNetV2
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. Pass the inputs to the base_model (note: using tf.keras.applications, EfficientNetV2 inputs don't have to be normalized)
x = base_model(inputs)
# Check data shape after passing it to base_model
print(f"Shape after base_model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(3, activation="softmax", name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# 10. Fit the model (we use less steps for validation so it's faster)
history = model_0.fit(train_data_leaves,
                                 epochs=5,
                                 steps_per_epoch=len(train_data_leaves),
                                 validation_data=train_data_leaves,
                                 # Go through less of the validation data so epochs are faster (we want faster experiments!)
                                 validation_steps=int(0.25 * len(train_data_leaves)),
                                 # Track our model's training logs for visualization later
                                 callbacks=[create_tensorboard_callback("transfer_learning", "10_percent_feature_extract")])

Shape after base_model: (None, 7, 7, 1280)
After GlobalAveragePooling2D(): (None, 1280)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extract/20240319-175408
Epoch 1/5
15/15 [==============================] - 47s 2s/step - loss: 0.6177 - accuracy: 0.8174 - val_loss: 0.2372 - val_accuracy: 1.0000
Epoch 2/5
15/15 [==============================] - 30s 2s/step - loss: 0.1609 - accuracy: 0.9978 - val_loss: 0.0891 - val_accuracy: 1.0000
Epoch 3/5
15/15 [==============================] - 32s 2s/step - loss: 0.0724 - accuracy: 1.0000 - val_loss: 0.0512 - val_accuracy: 1.0000
Epoch 4/5
15/15 [==============================] - 30s 2s/step - loss: 0.0454 - accuracy: 1.0000 - val_loss: 0.0333 - val_accuracy: 1.0000
Epoch 5/5
15/15 [==============================] - 34s 2s/step - loss: 0.0327 - accuracy: 1.0000 - val_loss: 0.0261 - val_accuracy: 1.0000


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_0)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)

In [ ]:
model_0.evaluate(test_data_leaves)

3/3 [==============================] - 5s 2s/step - loss: 0.0831 - accuracy: 1.0000


[0.08309602737426758, 1.0]

In [ ]:
# import numpy as np
# import requests
# from PIL import Image

# # Read the image file
# image_path = "D:\\CODDING\\GITHUB\\EES\\dataset\\mangodataset\\Gall Midge\\IMG_20211106_162102 (Custom).jpg"
# image = Image.open(image_path)

# # Resize the image to the required dimensions
# target_size = (224, 224)
# resized_image = image.resize(target_size)

# # Convert the image to a numpy array of floating-point numbers
# image_array = np.array(resized_image, dtype=np.float32)

# # Normalize the pixel values (optional)
# image_array /= 255.0

# # Add a batch dimension
# input_data = np.expand_dims(image_array, axis=0)

# # Prepare the input data in the required format (e.g., JSON)
# input_json = {
#     "instances": input_data.tolist()
# }

# # URL for the TensorFlow Serving REST API
# api_url = "http://localhost:8601/v1/models/plant_model/1:predict"

# # Send the POST request to the TensorFlow Serving API
# response = requests.post(api_url, json=input_json)

# # Check if the request was successful
# if response.status_code == 200:
#     # Get the predictions
#     predictions = response.json()

#     # Print the predictions
#     print(predictions)
# else:
#     print("Error:", response.text)


Error: {
    "error": "Malformed request: POST /v1/models/plant_model/1:predict"
}


In [ ]:
import tensorflow as tf

In [ ]:
# Path to the saved model directory
model_path = r'D:\CODDING\GITHUB\EES\plantify\saved_model\1'

# Load the model
model = tf.keras.models.load_model(model_path)

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('D:\CODDING\GITHUB\EES\dataset\guava dataset\Guava Dataset (Original Image)\Disease Free\Disease Free (7).jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

1/1 [==============================] - 0s 64ms/step


In [ ]:
result

array([[0.96581906, 0.01451117, 0.01966973]], dtype=float32)

In [ ]:
import numpy as np
import requests
from PIL import Image

# Read the image file
image_path = "D:\CODDING\GITHUB\EES\dataset\guava dataset\Guava Dataset (Original Image)\Disease Free\Disease Free (7).jpg"
image = Image.open(image_path)

# Resize the image to the required dimensions
target_size = (224, 224)
resized_image = image.resize(target_size)

# Convert the image to a numpy array of floating-point numbers
image_array = np.array(resized_image, dtype=np.float32)

# Normalize the pixel values (optional)
image_array /= 255.0

# Add a batch dimension
input_data = np.expand_dims(image_array, axis=0)

# Prepare the input data in the required format (e.g., JSON)
input_json = {
    "instances": input_data.tolist()
}
# http://localhost:8501/v1/models/plant_model/versions/1:predict
# URL for the TensorFlow Serving REST API
api_url = "http://localhost:8501/v1/models/plant_model/versions/1:predict"

# Send the POST request to the TensorFlow Serving API
response = requests.post(api_url, json=input_json)

# Check if the request was successful
if response.status_code == 200:
    # Get the predictions
    predictions = response.json()

    # Print the predictions
    print(predictions)
else:
    print("Error:", response.text)


{'predictions': [[0.0546299294, 0.705981195, 0.239388928]]}
